## Baseline-решение предсказания свойств флуоресцентных белков

### Подготовка к работе

In [152]:
! pip install scikit-learn pandas numpy 

In [153]:
from ast import literal_eval
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Загрузка данных

Загрузим данные из csv файла. Разделеним на обучающую и тестовую выборки (80/20).

In [ ]:
df = pd.read_csv("dataset.csv")
df_train, df_test = train_test_split(df, test_size=0.2, random_state=2009) # test_size - размер тестовой выборки

### KNN

Извлечем последовательности и целевые значения из датафрейма. Пишем только те у которых есть числовые данные в таргетах.

In [ ]:
def get_target(df, target):
    sequences_list = []
    target_list = []

    for idx, row in df.iterrows():
        if pd.notna(row[target]):
            if isinstance(row['sequence'], str):
                try:
                    seq = literal_eval(row['sequence'])
                except (SyntaxError, ValueError):
                    seq = np.fromstring(
                        row['sequence'].strip('[]'), 
                        sep=' ', 
                        dtype=np.float32
                    )
            else:
                seq = row['sequence']
            
            sequences_list.append(seq)
            target_list.append(row[target])

    return [np.array(sequences_list), np.array(target_list)] # [эмбеддинги, свойства]

Настроим KNN регрессию, найдем лучшее количества соседей (k) методом кросс-валидации.

In [ ]:
def regressor_knn(X_train, X_test, y_train, y_test):
    params = {'n_neighbors' : range(1, 30)}

    knn_regressor = KNeighborsRegressor()
    grid = GridSearchCV(knn_regressor, params, cv=5, scoring='neg_mean_squared_error') # MSE
    grid.fit(X_train, y_train)

    knn_regressor = KNeighborsRegressor(n_neighbors=grid.best_params_['n_neighbors'])
    knn_regressor.fit(X_train, y_train)
    predictions = knn_regressor.predict(X_test)

    return [np.sqrt(-grid.best_score_), grid.best_params_['n_neighbors'], knn_regressor, predictions] # метрики, гиперпараметры и предсказанные значения

Настроим KNN классификацию, найдем лучшее количество соседей (k) методом кросс-валидации.

In [ ]:
def classification_knn(X_train, X_test, y_train, y_test):
    params = {'n_neighbors' : range(1, 30)} # ищем k ближайщих соседей из диапазона 

    knn_classif = KNeighborsClassifier()
    grid = GridSearchCV(knn_classif, params, cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)

    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'] # метрики
    results = cross_validate(knn_classif, X_train, y_train, cv=5, scoring=scoring)

    knn_classif = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'])
    knn_classif.fit(X_train, y_train)
    predictions = knn_classif.predict(X_test)

    return [results, grid.best_score_, grid.best_params_['n_neighbors'], knn_classif, predictions] # метрики, гиперпараметры и предсказанные значения

Нпишем функции для предсказания свойств флуоресцентных белков.

In [ ]:
# Яркость
def knn_brightness():
    params_train = get_target(df_train, 'brightness')
    params_test = get_target(df_test, 'brightness')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Максимум длины волны испускания
def knn_em_max():
    params_train = get_target(df_train, 'em_max')
    params_test = get_target(df_test, 'em_max')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Максимум длины волны поглощения
def knn_ex_max():
    params_train = get_target(df_train, 'ex_max')
    params_test = get_target(df_test, 'ex_max')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Эффективность поглощения
def knn_ext_coeff():
    params_train = get_target(df_train, 'ext_coeff')
    params_test = get_target(df_test, 'ext_coeff')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Время жизни
def knn_lifetime():
    params_train = get_target(df_train, 'lifetime')
    params_test = get_target(df_test, 'lifetime')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Время созревания
def knn_maturation():
    params_train = get_target(df_train, 'maturation')
    params_test = get_target(df_test, 'maturation')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Устойчивость к кислотам
def knn_pka():
    params_train = get_target(df_train, 'pka')
    params_test = get_target(df_test, 'pka')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Стоксовский сдвиг
def knn_stokes_shift():
    params_train = get_target(df_train, 'stokes_shift')
    params_test = get_target(df_test, 'stokes_shift')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Квантовый выход
def knn_qy():
    params_train = get_target(df_train, 'qy')
    params_test = get_target(df_test, 'qy')

    return regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1])

# Агрегация
def knn_agg():
    params_train = get_target(df_train, 'agg')
    params_test = get_target(df_test, 'agg')

    return classification_knn(params_train[0], params_test[0], params_train[1], params_test[1])

### Результаты и метрики

Выведем полученные метрики.

In [ ]:
def metrics(results, target):
    if len(results) == 4:
        print(f'{target:<12} | RMSE: {results[0]}, k: {results[1]}')
    else:
        print(
            f'{target:<12} | accuracy: {results[0]['test_accuracy'].mean()}' + 
            f', precision: {results[0]['test_precision_weighted'].mean()}' + 
            f', recall: {results[0]['test_recall_weighted'].mean()}' +
            f', f1: {results[0]['test_f1_weighted'].mean()}'
        )

In [150]:
metrics(knn_brightness(),'brightness')
metrics(knn_ex_max(),'ex_max')
metrics(knn_em_max(),'em_max')
metrics(knn_ext_coeff(), 'ext_coeff')
metrics(knn_lifetime(), 'lifetime')
metrics(knn_maturation(), 'maturation')
metrics(knn_pka(), 'pka')
metrics(knn_stokes_shift(), 'stokes_shift')
metrics(knn_qy(), 'qy')

metrics(knn_agg(), 'agg')

brightness   | RMSE: 25.745918761587454, k: 5
ex_max       | RMSE: 46.881365072569494, k: 4
em_max       | RMSE: 38.164471063026305, k: 2
ext_coeff    | RMSE: 33260.04299675388, k: 5
lifetime     | RMSE: 1.2675000406190717, k: 24
maturation   | RMSE: 151.75877090035087, k: 5
pka          | RMSE: 1.1087124833060096, k: 7
stokes_shift | RMSE: 35.289969788204786, k: 18
qy           | RMSE: 0.22867770525347184, k: 3
agg          | accuracy: 0.7029221882432892, precision: 0.6745515232982797, recall: 0.7029221882432892, f1: 0.6770289846280668
